## Group1 Homwork - Feb 2

#### Yun LI, Hemin HE, Yi LIANG, Xiaoyi HAO, Yang LI, Yuwen CHE

### Exercise I

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

D:\anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
erf = pd.read_csv('https://github.com/QuantEcon/ShenzhenWinterCamp/raw/master/Jiao%20Shi/ER_predict.csv')
erf.head()

,Date,CNYUSD,RMB_REER,CHN PMI,USA PMI,CHN M1,USA M1,CHN M2,USA M2,CHN CPI inflation,USA CPI inflation,CHN_IBR,US_FFR
0,2005-07,8.1062,85.01,51.1,52.8,"97,700.00","1,365.00","277,000.00","6,498.40",1.8,3.2,1.1395,3.36
1,2005-08,8.0998,85.05,52.6,52.4,"99,400.00","1,376.50","281,300.00","6,522.50",1.3,3.6,1.1774,3.30
2,2005-09,8.0920,85.29,55.1,56.8,"101,000.00","1,363.10","287,000.00","6,555.00",0.9,4.7,1.1675,3.57
3,2005-10,8.0845,86.39,54.1,57.2,"101,800.00","1,365.10","287,600.00","6,582.30",1.2,4.3,1.2442,3.73
4,2005-11,8.0804,87.75,54.1,56.7,"104,100.00","1,373.20","292,400.00","6,623.00",1.3,3.5,1.2049,3.99


In [3]:
# compute the change in the exchange rate: take log and first difference to compute depreciation (dep)
erf['lner'] = np.log(erf[['CNYUSD']])
erf['dep'] = 100*erf['lner'].diff(periods = 1) 

In [4]:
# Create a new column to reindex depreciation - why?
erf['dep_1'] = 0
size_sample = 149

for i in range(size_sample):
    erf.loc[i,'dep_1'] = erf.loc[i+1,'dep']
    
erf.loc[size_sample,'dep_1'] = erf.loc[0,'dep']

In [5]:
erf['int_diff'] = erf['CHN_IBR'] - erf['US_FFR'] # The interest difference between countries

In [6]:
erf['const'] = 1 # create column of constant

reg_int = sm.OLS(endog=erf['dep_1'], exog=erf[['const', 'int_diff']], missing='drop')
results = reg_int.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  dep_1   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     4.998
Date:                Fri, 02 Feb 2018   Prob (F-statistic):             0.0269
Time:                        20:32:17   Log-Likelihood:                -148.45
No. Observations:                 149   AIC:                             300.9
Df Residuals:                     147   BIC:                             306.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1985      0.059     -3.355      0.0

In [7]:
# Int Model

size_sample = 149
size_subsample = 60

# How many regressions can we run? How many forecast errors can we compute?
fcst_error_int = np.zeros(size_sample - size_subsample)
fcst_prediction_int = np.zeros(size_sample - size_subsample)

for i in range (size_sample - size_subsample):
    reg_intsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'int_diff']], missing='drop')
    results_sub = reg_intsub.fit()
    fcst_error_int[i] = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_prediction_int[i] = results_sub.predict(exog = [1, erf.loc[size_subsample+i,'int_diff']])

In [8]:
# Taylor-Rule model

# inflation differential
erf['pi_diff'] = erf['CHN CPI inflation'] - erf['USA CPI inflation']

fcst_tr = np.zeros(size_sample - size_subsample)
fcst_error_tr = np.zeros(size_sample - size_subsample)
fcst_prediction_tr = np.zeros(size_sample - size_subsample)

for i in range (size_sample - size_subsample):
    # Create a column to store the output gap difference and update it in each loop
    y_chn = erf['CHN PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'CHN PMI'])
    y_us = erf['USA PMI'] - np.mean(erf.loc[i:size_subsample-1+i, 'USA PMI'])
    erf['y_diff'] = y_chn - y_us
    
    reg_trsub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'pi_diff','y_diff']], missing='drop')
    results_sub = reg_trsub.fit()
    
    fcst_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff']]])
    fcst_error_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff']]]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_prediction_tr[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff']]])

In [9]:
pcdf_int = ((fcst_error_int*fcst_prediction_int >= 0 ).sum())/len(fcst_error_int)
pcdf_tr = ((fcst_error_tr*fcst_prediction_tr >= 0 ).sum())/len(fcst_error_tr)
pcdf_rw = ((erf.loc[60:148, 'dep_1']*erf.loc[60:148, 'dep']>=0).sum())/89

print(pcdf_int)
print(pcdf_tr)
print(pcdf_rw)

0.573033707865
0.606741573034
0.61797752809


### Exercise II.a


The real effective exchange rate (REER) is the weighted average of a country's currency relative to an index or basket of other major currencies, adjusted for the effects of inflation. The weights are determined by comparing the relative trade balance of a country's currency against each country within the index. This exchange rate is used to determine an individual country's currency value relative to the other major currencies in the index, such as the U.S. dollar, Japanese yen and the euro.

### Exercise II.b

In [10]:
# The Taylor Rule model with REER

fcst_reer = np.zeros(size_sample - size_subsample)
fcst_error_reer = np.zeros(size_sample - size_subsample)
fcst_prediction_reer = np.zeros(size_sample - size_subsample)

for i in range (size_sample - size_subsample):
    
    reg_reersub = sm.OLS(endog=erf.loc[i:size_subsample-1+i,'dep_1'], exog=erf.loc[erf.index[i:size_subsample+i],['const', 'pi_diff','y_diff','RMB_REER']], missing='drop')
    results_sub = reg_reersub.fit()
    
    fcst_reer[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff','RMB_REER']]])
    fcst_error_reer[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff','RMB_REER']]]) - erf.loc[size_subsample+i, 'dep_1']
    fcst_prediction_reer[i] = results_sub.predict(exog = [erf.loc[size_subsample+i,['const', 'pi_diff','y_diff','RMB_REER']]])
    
print(results_sub.summary())

                            OLS Regression Results                            
Dep. Variable:                  dep_1   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     2.145
Date:                Fri, 02 Feb 2018   Prob (F-statistic):              0.105
Time:                        20:32:19   Log-Likelihood:                -73.014
No. Observations:                  60   AIC:                             154.0
Df Residuals:                      56   BIC:                             162.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1249      2.316     -1.781      0.0

The sign of the coefficient of REER is positive and statistically significant at 90% confidence level. The sign is not consistent with our intuition, since when REER is larger, the strength of Chinese Yuan is higher, and so that the exchange rate should fall.

In [11]:
rmse_reer = np.sqrt(np.mean(fcst_error_reer**2))
rmse_tr = np.sqrt(np.mean(fcst_error_tr**2))

print(rmse_tr)
print(rmse_reer)

0.771097843365
0.7605718598


The new model produces a smaller RMSE than the original Taylor Rule model.